# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*


## Практикум 4. Управление браузером с Selenium: введение

Так как в этом практикуме мы будем из Python запускать окна браузера – эпиграф!

>*– Итак! За главным звуком засилья застойного затишья боярского застолья ворвался грозный стук топора, которым Петр рубил окно в Европу.*<br>
*– Где рубил?*<br>
*– В стене!*<br>
*– В какой?*<br>
*– В какой стене можно рубить топором?!!! Конечно, в деревянной!!!*<br>
*– А дверь прорубил?*<br>
*– Нет!*<br>
*– Значит, они так в окно и лазают...*<br>
*– Кто?!*<br>
*– Ну, петровцы...*<br>
<br>
*Диалог Мыши и Попугая (немного сокращён) из радиопьесы «Алиса в Стране чудес» (1976)*



### Установка и подготовка к работе

Библиотека Selenium – библиотека для управления браузером с помощью Python. Она позволяет запускать браузер и имитировать действия пользователя в нём. 

Зачем это может понадобиться? Во-первых, такое «встраивание» в браузер в большинстве случаев позволяет решить проблему с возникающими капчами и иными ограничениями, так как настройки сайта не распознают, что запрос к исходному коду страницы производится автоматически. Во-вторых, необходимость имитации действий в браузере неизбежно возникает при обработке динамических веб-страниц, где некоторые элементы (окна, графики, таблицы) появляются только при определенных действиях пользователя, например, при скроллинге или наведении мышкой. В-третьих, библиотека может быть полезна в случаях, если доступ к API сайта или базы данных получить довольно сложно, но при работе в браузере информация доступна (в таком случае можем залогиниться через Python как пользователь и потихоньку выгружать данные).

Установим библиотеку:

In [ ]:
!pip install selenium

**NB.** Если библиотека уже установлена, команда выше вернет большой текст с сообщениями вида `Requirement already satisfied`, если не помните, когда успели ее установить, лучше принудительно обновить версию на всякий случай:

    !pip install selenium --upgrade
    
Зачем? В новой версии библиотека `selenium` при первом запуске браузера из Python сама скачает подходящий драйвер для соединения с браузером и будет использовать его в дальнейшем, а в старых версиях будет искать файл с драйвером на компьютере и выдавать ошибку.

Сначала импортируем библиотеку полностью, проверим, что всё идёт по плану:

In [1]:
import selenium

Теперь импортируем отдельные компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запустим браузер Chrome средствами Selenium (в модуле есть функции для разных браузеров, но лучше всего работать с Chrome или Firefox):

In [3]:
br = wd.Chrome()

Если вы исполняете код выше первый раз, это может занять некоторое время, так как функция `Chrome()` сначала скачает и установит подходящую версию драйвера для Chrome, который будет обеспечивать связь между Selenium и той версией браузера Chrome, которая установлена у вас на компьютере. В последующие разы всё должно происходить быстрее.

В результате запуска строки выше должно открыться новое окно браузера Chrome. Оно пустое и, скорее всего, с всплывающим предупреждением о том, что браузером управляет автоматизированное тестовое ПО. Это нормально, мы «имитировали» открытие браузера от лица пользователя, теперь, если мы будем в этом окне переходить по ссылкам на сайты, сайты не будут воспринимать запросы как автоматические (в отличие от запросов через модуль `requests`, из-за автоматических или слишком быстрых запросов, собственно, могут возникать те же капчи).

Открывшееся окно браузера в течение работы закрывать нельзя. Python будет там выполнять действия, а мы будем их отслеживать со стороны.
Приступим к работе.

Открываем новое окно на всю ширину экрана на случай, если какие-то элементы в маленьком окне будут мешать или накладываться друг на друга (так бывает с рекламой, всплывающими окнами и подобным):

In [4]:
br.maximize_window()

### Поиск элементов на странице

Начнём с простого примера – сайта книжного магазина «Библио-Глобус». Вообще этот сайт можно парсить и без Selenium, он пока не блокирует автоматические запросы, но зато другие сайты магазинов или онлайн-кинотеатров с похожей структурой умеют это делать. 

Отправим запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [5]:
br.get("https://www.biblio-globus.ru/")

В окне должна открыться главная страница сайта. Найдём на странице поле для поиска интересующего товара. Для этого нам понадобится метод `.find_element()`, он применяется к объекту *WebDriver* (у нас `br`). Вообще в Selenium есть два метода для поиска, по аналогии с методами `.find()` и `.find_all()` в BeautifulSoup:

* `.find_element()` – поиск одного элемента, возвращает один результат, если подходящих элементов несколько, возвращается первый;
* `.find_elements()` – поиск нескольких элементов, возвращает список результатов.

Искать будем по id объекта, его можем найти в исходном коде страницы или через инструменты разработчика.

In [7]:
search = br.find_element(By.ID, "SearchBooks")

В инсрументах разработчика id объекта отображается с `#`, классы с `.`, в нём же можно запросить более сложный запрос `XPATH`, но об этом поговорим позже. Сейчас в `search` у нас хранится объект Selenium, к которому можно применять различные методы. Так как мы нашли поле для поиска, которое можно заполнять, мы воспользуемся методом `.send_keys()`, который введёт в это поле текст. Запросим книги по Python:

In [8]:
search.send_keys("python")

В открытом окне должен появиться текст и выпадающее меню с названиями книг. Магия! 

**Примечание.** Если мы нашли объект некорректно, например, не само поле (в HTML обычно с тэгом `<input>`), а рамочку вокруг него или раздел, внутри которого это поле находится, то есть те элементы, которые не подразумевают интерактива в виде ввода значений, в ответ на `.send_keys()` мы получим ошибку `element not interactable`.

Для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

In [9]:
search.send_keys(Keys.ENTER)

Отлично! Теперь нужно поработать с результатами поиска. Предположим, что мы пишем код – основу для какой-нибудь программы или приложения, которая позволяет пользователю по ключевым словам выполнить поиск и отсортировать результаты по какому-нибудь критерию, доступному на сайте. Например, по цене, от меньшей к большей. 

Найдём выпадающее меню с нужными опциями на странице по названию тэга.

In [10]:
options = br.find_element(By.TAG_NAME, "select")

По аналогии с предыдущим примером заполнения ввода поля, выбор опций в обычных выпадающих меню с тэгом `<select>` можно осуществить с помощью того же метода `.send_keys()`.

In [11]:
options.send_keys(" Сначала дешевле ") # обратите внимание на пробелы (так в исходном коде)

Отлично! Давайте ещё найдём поле для галочки «В наличии» и кликнем на него!

In [12]:
tick = br.find_element(By.CLASS_NAME, "custom-control-label")
tick.click()

Технически, нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью BeautifulSoup. 

In [13]:
html = br.page_source

Попробуем найти карточки с найденными товарами!

In [14]:
from bs4 import BeautifulSoup

In [15]:
# ну, а тут как всегда,
# вот сами карточки, в следующем практикуме продолжим

soup = BeautifulSoup(html)

In [18]:
# имеют класс card
# первый элемент – лишний (там не товар, но класс тоже card)

cards = soup.find_all("div", class_ = "card")[1:]